# 세이지메이커 점프스타트에서 Llama 2 모델 실행

In [ ]:
model_id = "meta-textgeneration-llama-2-7b-f"

In [ ]:
model_version = "2.*"

## 모델 배포

***
이제 세이지메이커 점프스타트를 활용하여 모델을 배포할 수 있습니다. 성공적인 배포를 위해서는 모델의 배포 메서드에서 `accept_eula` 인수를 `True`로 수동으로 변경해야 합니다.
***

In [ ]:
from sagemaker.jumpstart.model import JumpStartModel

model = JumpStartModel(model_id=model_id, model_version=model_version)

In [ ]:
predictor = model.deploy(accept_eula=True)

## 엔드포인트 호출

***
### 지원되는 매개변수

***
이 모델은 추론 시 여러 매개변수를 지원합니다. 포함되는 매개변수는 다음과 같습니다.

* **max_length:** 모델이 출력 길이(입력 컨텍스트 길이 포함)가 `max_length`에 도달할 때까지 텍스트를 생성합니다. 지정하는 경우, 양의 정수여야 합니다.
* **max_new_tokens:** 모델이 출력 길이(입력 컨텍스트 길이 제외)가 `max_new_tokens`에 도달할 때까지 텍스트를 생성합니다. 지정하는 경우, 양의 정수여야 합니다.
* **num_beams:** 탐욕 검색에 활용되는 빔의 수입니다. 지정하는 경우, `num_return_sequences`보다 크거나 같은 정수여야 합니다.
* **no_repeat_ngram_size:** 모델이 `no_repeat_ngram_size`의 단어 시퀀스가 출력 시퀀스에서 반복되지 않도록 합니다. 지정하는 경우, 1보다 큰 양의 정수여야 합니다.
* **temperature:** 출력의 무작위성을 제어합니다. 높은 온도는 확률이 낮은 단어로 구성된 출력 시퀀스를 생성하며, 낮은 온도는 확률이 높은 단어로 구성된 출력 시퀀스를 생성합니다. `temperature` -> 0일 경우 탐욕적 디코딩이 됩니다. 지정하는 경우, 양의 실수여야 합니다.
* **early_stopping:** True일 경우, 모든 빔 가설이 문장 종료 토큰에 도달하면 텍스트 생성이 종료됩니다. 지정하는 경우, 불리언이어야 합니다.
* **do_sample:** True일 경우, 다음 단어를 확률에 따라 샘플링합니다. 지정하는 경우, 불리언이어야 합니다.
* **top_k:** 텍스트 생성의 각 단계에서, `top_k` 가장 가능성이 높은 단어 중에서 샘플링합니다. 지정하는 경우, 양의 정수여야 합니다.
* **top_p:** 텍스트 생성의 각 단계에서, 누적 확률이 `top_p`인 단어의 최소 집합에서 샘플링합니다. 지정하는 경우, 0과 1 사이의 실수여야 합니다.
* **return_full_text:** True일 경우, 입력 텍스트가 생성된 텍스트의 일부로 포함됩니다. 지정하는 경우, 불리언이어야 합니다. 기본값은 False입니다.
* **stop**: 지정하는 경우, 문자열 목록이어야 합니다. 지정된 문자열 중 하나가 생성되면 텍스트 생성이 멈춥니다.

엔드포인트를 호출할 때 위의 매개변수 중 일부를 지정할 수 있습니다. 다음은 이러한 인수를 활용하여 엔드포인트를 호출하는 예입니다.

**참고**: `max_new_tokens`가 정의되지 않은 경우, 모델은 최대 총 토큰 수인 4K까지 생성할 수 있습니다. 이는 엔드포인트 쿼리 타임아웃 오류를 유발할 수 있으므로 가능한 경우 `max_new_tokens`를 설정하는 것이 좋습니다. 7B, 13B 및 70B 모델의 경우, 총 토큰 수를 4K 이하로 유지하면서 각각 1500, 1000, 500을 초과하지 않도록 `max_new_tokens`를 설정하는 것이 좋습니다.

**참고**: 이 모델은 'system', 'user', 'assistant' 역할만 지원하며, 'system'으로 시작하여 'user'와 번갈아 가며 (u/a/u/a/u...) 진행됩니다.
***

### 프롬프트 예시
***
이 섹션의 예시는 대화형 다이얼로그를 프롬프트 입력으로 텍스트 생성을 수행하는 방법을 보여줍니다. 페이로드 예시는 `JumpStartModel` 객체에서 프로그래밍 방식으로 검색됩니다.

Llama-2 채팅 모델에 대한 입력 메시지는 다음 형식을 가져야 합니다. 모델은 'system', 'user', 'assistant' 역할만 지원하며, 'system'으로 시작하여 'user'와 번갈아 가며 (u/a/u/a/u...) 진행됩니다. 마지막 메시지는 'user'로부터 와야 합니다. 간단한 사용자 프롬프트는 다음과 같을 수 있습니다.

```
<s>[INST] {user_prompt} [/INST]
```

다음 구문으로 시스템 프롬프트를 추가할 수 있습니다.

```
<s>[INST] <<SYS>>
{system_prompt}
<</SYS>>

{user_prompt} [/INST]

```

마지막으로, 모든 이전 사용자 프롬프트와 어시스턴트 응답을 포함하여 모델과의 대화형 상호작용을 수행할 수 있습니다.

```
<s>[INST] <<SYS>>
{system_prompt}
<</SYS>>

{user_prompt_1} [/INST] {assistant_response_1} </s><s>[INST] {user_prompt_1} [/INST]
```
***

In [ ]:
example_payloads = model.retrieve_all_examples()
example_payloads[0].body

In [ ]:
response = predictor.predict(example_payloads[0].body, custom_attributes="accept_eula=true")
    
print(response[0]["generation"]["content"])

***
이전 예제 페이로드에서는 활용되지 않았지만, 다음 유틸리티 함수로 Llama-2 모델에 메시지를 형식화할 수 있습니다.
***

In [ ]:
payload = {
    'inputs': [
        [{
            'role': 'system',
            'content': 'Always answer with Haiku.'
         },
         {
            'role': 'user',
            'content': 'I am going to Paris, what should I see?'
         }]            
    ],
    'parameters': {
        'max_new_tokens': 512, 'top_p': 0.9, 'temperature': 0.6
    }
}

response = predictor.predict(payload, custom_attributes="accept_eula=true")

print(response[0]["generation"]["content"])

## 엔드포인트 정리

In [1]:
# # 세이지메이커 엔드포인트 삭제
# predictor.delete_model()
# predictor.delete_endpoint()